<a href="https://colab.research.google.com/github/sadnow/AnimationKit-AI_Upscaling-Interpolation_RIFE-RealESRGAN/blob/testing/TESTING_AnimationKit_Rife_RealESRGAN_Upscaling_Interpolation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AnimationKit AI - Upscaling & Interpolation using Real-ESRGAN+RIFE

**ALPHA 2: Released 9/4/21**

New features:

- Deflickering (P3)
- Upscaling from individual frames (P2)
- Target length (in seconds) for RIFE interpolation

**-WARNING: THIS IS THE TESTING BRANCH! Try the main branch for stability-**

Main branch can be found here: https://github.com/sadnow/AnimationKit-AI_Upscaling-Interpolation_RIFE-RealESRGAN


Credits: Motion smoothing conceived from "Zoom animation processing and motion interpolation" added by https://twitter.com/unltd_dream_co. This part of the script uses [RIFE real-time video interpolation](https://github.com/hzwer/arXiv2020-RIFE) to smooth out the resulting video. 

Upscaling uses Real-ESRGAN (https://github.com/xinntao/Real-ESRGAN). A demo notebook for static images can be found here: https://colab.research.google.com/drive/1k2Zod6kSHEvraybHl50Lys0LerhyTMCo?usp=sharing. The demo was based on the following paper: of our paper [''Real-ESRGAN: Training Real-World Blind Super-Resolution with Pure Synthetic Data''](https://arxiv.org/abs/2107.10833).

<img src="https://raw.githubusercontent.com/xinntao/Real-ESRGAN/master/assets/teaser.jpg" width="100%">

Special thanks to @sportsracer48 and everyone on his Discord. If you want closed beta access to the best VQGAN animation notebook on the planet, check out https://www.patreon.com/sportsracer48

---

Testing notes:

Many ffmpeg examples at https://docs.google.com/document/d/12X_2YoCnPPN7B3OsgX39aYyRF8OF-TVStkFTkKhWrx4/edit

Python modules in Colab: https://medium.com/analytics-vidhya/importing-your-own-python-module-or-python-file-into-colab-3e365f0a35ec

Colab Markdown guide: https://colab.research.google.com/notebooks/markdown_guide.ipynb?hl=es-ni#scrollTo=Lhfnlq1Surtk



In [ ]:
# @title Licensed under the MIT License

# Copyright (c) 2021 Katherine Crowson

# Permission is hereby granted, free of charge, to any person obtaining a copy
# of this software and associated documentation files (the "Software"), to deal
# in the Software without restriction, including without limitation the rights
# to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
# copies of the Software, and to permit persons to whom the Software is
# furnished to do so, subject to the following conditions:

# The above copyright notice and this permission notice shall be included in
# all copies or substantial portions of the Software.

# THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
# IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
# FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
# AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
# LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
# OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
# THE SOFTWARE.

In [ ]:
!nvidia-smi

Mon Sep  6 17:56:17 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.63.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   34C    P0    22W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# P0: Mount Google Drive & Install Libraries

In [3]:
#I'm leaving these outside AnimationKit.py for transparency
import os, sys, re
from multiprocessing import Process

def need_dir(a):
    import os.path
    from os import path
    if not path.exists(a):
      print("Creating"+a+"...")
      !mkdir $a

  def need_file(a):
    import os.path
    from os import path
    if not path.isfile(a):
      print("Creating"+a+"...")
      !mkdir $a

  def check_dir(a):
    import os.path
    from os import path
    if not path.exists(a):
      print("ERROR: "+a+" does not exist!")

  def check_file(a):
    import os.path
    from os import path
    if not path.isfile(a):
      print("ERROR: "+a+" does not exist!")


class AnKit:

  def sortFrames(sourceframes, destframes):  #takes frames from input folder, moves to init_frame_storage
    check_dir(sourceframes)
    need_dir(destframes)
    %cd $sourceframes
    print("Copying frames to "+destframes+" for processing...")
    !find -maxdepth 1 -name '*.png' -print0 | xargs -0 cp -t $destframes
    print("Finished copying frames to "+destframes+".")

  def splitFrames(sourcefile, destframes):
    import os.path
    from os import path
    check_file(sourcefile)
    need_dir(destframes)
    !ffmpeg -r 1 -i $sourcefile -r 1 $destframes/frame%d.png

  def rebuildFrames():
  %cd /content/Real-ESRGAN/results/
  if input_mp4:
    !ffmpeg -r $target_fps -i 'out_%d_out.png' $output_path_mp4   # x264 video
  else:
    !ffmpeg -r $target_fps -i 'out_%d_out.png' $output_path_mp4
  
  def runUpscale(esrgan,mpath,scale,input,output):
    check_dir(input)
    need_dir(output)
    %cd $esrgan
    print(mpath)
    scale = int(scale)
    !python inference_realesrgan.py --model_path $mpath --netscale $scale --input $input --output $output

  def installPhase():
    %cd /content/
    anKit.installESRGAN()
    %cd /content/
    anKIT.installRIFE()
    %cd /content/

  def install_ESRGAN():
    print("Installing libraries for Real-ESRGAN upscaling.")
    !git clone https://github.com/xinntao/Real-ESRGAN.git
    %cd Real-ESRGAN
    !pip -q --no-python-version-warning install basicsr
    !pip -q --no-python-version-warning install facexlib
    !pip -q --no-python-version-warning install gfpgan
    !pip -q --no-python-version-warning install -r requirements.txt
    !python setup.py develop
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth -P experiments/pretrained_models
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.2.4/RealESRGAN_x4plus_anime_6B.pth -P experiments/pretrained_models
    !wget https://github.com/xinntao/Real-ESRGAN/releases/download/v0.2.1/RealESRGAN_x2plus.pth -P experiments/pretrained_models
    print("Finished Installing libraries for Real-ESRGAN upscaling.")
    #
    def installRIFE():
    print("Installing libraries for RIFE motion smoothing.")
    !git clone https://github.com/hzwer/Practical-RIFE Practical-RIFE
    !gdown --id 1O5KfS3KzZCY3imeCr2LCsntLhutKuAqj
    !7z e /content/Practical-RIFE/RIFE_trained_model_v3.8.zip
    #!7z e /content/RIFE_trained_model_v3.8.zip
    !mkdir /content/Practical-RIFE/train_log
    !mv *.py /content/Practical-RIFE/train_log/
    !mv *.pkl /content/Practical-RIFE/train_log/
    %cd /content/Practical-RIFE/
    !gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc #useless - mp4 demo
    !pip3 install -r requirements.txt
    print("Finsihed Installing libraries for RIFE motion smoothing.")
    #
    def installOldRIFE():
    print("Installing libraries for RIFE motion smoothing.")
    !git clone https://github.com/hzwer/arXiv2020-RIFE RIFE
    !gdown --id 1wsQIhHZ3Eg4_AfCXItFKqqyDMB4NS0Yd
    !7z e RIFE_trained_model_HDv2.zip
    !mkdir /content/RIFE/train_log
    !mv *.pkl /content/RIFE/train_log/
    %cd /content/RIFE/
    !gdown --id 1i3xlKb7ax7Y70khcTcuePi6E7crO_dFc
    !pip3 install -r requirements.txt
    print("Done.")
    print("Finsihed Installing libraries for RIFE motion smoothing.")
    
###



###############################################################

#
#If RIFE (P2) stops for no reason, go to Runtime>Change Runtime type and set the notebook to "high memory"

#Params
mount_google_drive = True #@param {type:"boolean"}

#Mount Google Drive
if mount_google_drive:
  from google.colab import drive
  drive.mount('/content/drive')

#install dependencies
anKit.installPhase()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


AttributeError: ignored

# P1: Real-ESRGAN Video Upscaling (Somewhat Experimental)

Planned additions: Deflickering, faces option, tiles option

In [ ]:
###



###

#############################################################

###PARAMS

#frame_rate = "23.976" #@param {type:"string"}
#start_time = "00:00:00.000"
#end_time = "00:00:05.000" #@param {type:"string"}
#@markdown #Input settings
#@markdown 
input_mp4 = True #@param {type:"boolean"}
mp4_filename='/content/drive/MyDrive/VQLIPSE/videos/2uncleIroh.mp4' #@param {type:"string"}
#@markdown ---
input_frame_folder='/content/drive/MyDrive/VQLIPSE/images_out/2uncleIroh' #@param {type:"string"}
#@markdown If you checkboxed `input_mp4`, `input_frame_folder` will be ignored (and vice-versa).

#@markdown
#@markdown ---
#@markdown #Output settings
output_path_mp4='/content/drive/MyDrive/VQLIPSE/videos/2uncleIroh_KIT.mp4' #@param {type:"string"}
target_fps='180'#@param {type:"string"}
model_path='/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth' #@param ['/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus_anime_6B.pth','/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth','/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x2plus.pth'] {type:"string"}
scale_value="4" #@param [2, 4] {type:"string"}

#@markdown `scale_value` should match the model name, eg 4xplus would use scale value of 4
#@markdown Run RIFE below to fill in empty or duplicate frames. (Double, quadruple etc the length of your video)

###############################################################
init_frame_storage = "/content/init_frame_storage"
output_frame_storage = "/content/"
esrgan_inference_path = "/content/Real-ESRGAN/inference_realesrgan.py"
processed_frame_output = "/content/upscaled_frame_storage"
###############################################################
%cd /content/
print("Cleaning up from last run...")
!rm -rf "/content/Real-ESRGAN/results"
!rm -rf $init_frame_storage


if input_mp4:
  splitFrames(mp4_filename,init_frame_storage) #split mp4 into frames
else:
  sortFrames(input_frame_folder,init_frame_storage)  #duplicates frames to init_frame_storage
  #not necessary if input_mp4 checked bcuz splitFrames() saves as %d

%cd /content/
runUpscale(esrgan_inference_path,model_path,scale_value,init_frame_storage,processed_frame_output)
rebuildFrames()

/content
Cleaning up from last run...
Creating/content/init_frame_storage...
ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --e

In [ ]:
def runUpscale(esrgan,mpath,scale,input,output):
  #import os
  #from google.colab import files
  #import shutil
  check_dir(input)
  need_dir(output)
  %cd $esrgan
  print(mpath)
  scale = int(scale)
  !python inference_realesrgan.py --model_path $mpath --netscale $scale --input $input --output $output

def rebuildFrames():
  #re-encode video from frames
  #results = '/content/Real-ESRGAN/results'
  #!ffmpeg -r 1/5 -pattern_type glob -i '*.png' -c:v libx264 /content/output.mp4   # x264 video
  %cd /content/Real-ESRGAN/results/
  #!ffmpeg -r $target_fps -pattern_type glob -i '*.png' -c:v libx264 $output_path_mp4   # x264 video
  if input_mp4:
    #!ffmpeg -r $target_fps -pattern_type glob -i '*.png' -c:v libx264 $output_path_mp4   # x264 video
    !ffmpeg -r $target_fps -i 'out_%d_out.png' $output_path_mp4   # x264 video

  else:
    !ffmpeg -r $target_fps -i 'out_%d_out.png' $output_path_mp4



mp4_filename="/content/drive/MyDrive/VQLIPSE/videos/2uncleIroh.mp4" #@param {type:"string"}
init_frame_storage = "/content/init_frame_storage"
output_frame_storage = "/content/output_frame_storage"
esrgan_inference_path = "/content/Real-ESRGAN/"
input_mp4 = False #@param {type:"boolean"}
input_frame_folder="/content/drive/MyDrive/VQLIPSE/images_out/2uncleIroh" #@param {type:"string"}
model_path="/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth" #@param ['/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus_anime_6B.pth','/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth','/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x2plus.pth'] {type:"string"}
scale_value="4" #@param [2, 4] {type:"string"}
processed_frame_output = "/content/upscaled_frame_storage"


#runUpscale(esrgan_inference_path,model_path,scale_value,init_frame_storage,processed_frame_output)







/content/Real-ESRGAN
/content/Real-ESRGAN/experiments/pretrained_models/RealESRGAN_x4plus.pth
Testing 0 frame1
Testing 1 frame10
Testing 2 frame100
Testing 3 frame1000
Testing 4 frame1001
Testing 5 frame1002
Testing 6 frame1003
Testing 7 frame1004
Testing 8 frame1005
Testing 9 frame1006
Testing 10 frame1007
Testing 11 frame1008
Testing 12 frame1009
Testing 13 frame101
Testing 14 frame1010
Testing 15 frame1011
Testing 16 frame1012
Testing 17 frame1013
Testing 18 frame1014
Testing 19 frame1015
Testing 20 frame1016
Testing 21 frame1017
Testing 22 frame1018
Testing 23 frame1019
Testing 24 frame102
Testing 25 frame1020
Testing 26 frame1021
Testing 27 frame1022
Testing 28 frame1023
Testing 29 frame1024
Testing 30 frame1025
Testing 31 frame1026
Testing 32 frame1027
Testing 33 frame1028
Testing 34 frame1029
Testing 35 frame103
Testing 36 frame1030
Testing 37 frame1031
Testing 38 frame1032
Testing 39 frame1033
Testing 40 frame1034
Testing 41 frame1035
Testing 42 frame1036
Testing 43 frame1037
T

# P2 Motion interpolation (Practical-RIFE Smoothing)

"(...) (O)utput can be a bit wobbly, so video interpolation can be used to smooth things out a bit."

New: Type in the number of seconds you want your clip to be!

Use target_scale to speed up RIFE. A value of 0.5 would make a 4k video 2k.


In [ ]:
def detect_fps():
  import re
  fps_ffprobe = !ffprobe -v error -select_streams v -of default=noprint_wrappers=1:nokey=1 -show_entries stream=avg_frame_rate $input_path
  fps_unfinished = [str(i) for i in fps_ffprobe] # Converting integers into strings
  fps_unfinishedTwo = str("".join(fps_unfinished)) # Join the string values into one string
  numbers = re.findall('[0-9]+', fps_unfinishedTwo)
  newNum = numbers[0:1]
  strings = [str(integer) for integer in newNum]
  a_string = "".join(strings)
  fps = int(a_string)
  #print("Detected FPS is",fps)
  return fps
#

def detect_duration():
  import re
  duration_ffprobe = !ffprobe -v error -select_streams v:0 -show_entries stream=duration -of default=noprint_wrappers=1:nokey=1 $input_path
  duration_unfinished = [str(i) for i in duration_ffprobe] # Converting integers into strings
  duration_unfinishedTwo = str("".join(duration_unfinished)) # Join the string values into one string
  numbers = re.findall('[0-9]+', duration_unfinishedTwo)
  newNum = numbers[0:1]
  strings = [str(integer) for integer in newNum]
  a_string = "".join(strings)
  duration = int(a_string)
  #print("Detected duration INTEGER (in seconds) is",duration)
  return duration
#

def exp_calc():
  import numpy as np
  a = measured_fps * measured_duration
  b = target_fps * length_in_seconds
  c = b / a
  l = np.log(c) / np.log(2)
  print("Un-rounded --exp is",l)
  x = round(l)
  print("Rounding up to an --exp of ",x)
  return x

%cd /content/Practical-RIFE/
# @title Settings
#input_path='/content/output.mp4' #@param {type:"string"}
input_path='/content/drive/MyDrive/VQLIPSE/videos/2uncleIroh_KIT.mp4' #@param {type:"string"}

#autodetect fps


target_fps=180#@param {type:"integer"}
length_in_seconds=60#@param {type:"integer"}
target_scale=0.5#@param{type:"slider", min:0.1, max:1, step:0.1}
#length_multiplier=3#@param {type:"integer"}
#@markdown
#@markdown
#@markdown ---
#@markdown Note: Your output will look like `original_2x_60fps.mp4` and can be found in your `input_path`.
#@markdown
#@markdown Tip: Save as a high FPS (eg 999) if you plan on doing heavy editing (that way you don't need to re-process at the end.)


measured_fps = detect_fps()
print("Detected average FPS of",input_path,"is",measured_fps)
measured_duration = detect_duration()
print("Detected duration INTEGER (in seconds) is",measured_duration)
exp_value = exp_calc()
print("Warning: Target duration currently rounds to the closest integer.")
!python3 /content/Practical-RIFE/inference_video.py --fps=$target_fps --exp=$exp_value --video=$input_path --scale $target_scale
%cd /content/

# P3: Deflickering & x265 Compression (Experimental, Very Fast)

Todo: Add option for "Auto-pick path" (for people who ran the above step)

Sort of like Handbrake - good for large filesizes. Can turn 500mb files into 200mb files with negligable quality loss. Deflickering is good for deflickering stopmotions, VQGAN_CLIP animations, etc.

Also, too high of compressions might require a decent local GPU to view. Will add h264 toggle in the future~



In [ ]:
def runFF():
  !ffmpeg -i $compress_path $visual_effects -c:v hevc_nvenc -rc vbr -cq $constant_quality -qmin $constant_quality -qmax $constant_quality -b:v 0 $compress_path$outputStr
  #top worked before upgrading ffmpeg


compress_path='/content/drive/MyDrive/VQLIPSE/videos/2blackhole_AnKIT_4X_120fps.mp4' #@param {type:"string"}
outputStr = '_tblend6.mp4' #@param {type:"string"}
constant_quality=35#@param {type:"slider", min:20, max:50, step:1}
#animation_tune_toggle = True #@param {type:"boolean"}
#@markdown Default `constant_quality` is `27`, which scrunched a 500mb 40 min video into ~200mb. Higher values = lower filesize, lower quality
#################
#@markdown ---
enable_visual_effects = True #@param {type:"boolean"}
#@markdown Disable visual_effects if you don't have any visual effects checked-
deflicker_on = False #@param {type:"boolean"}
deflicker_avg_frames=100#@param {type:"slider", min:2, max:129, step:1}
mpdecimate_on = False #@param {type:"boolean"}
minterpolate_on = False #@param {type:"boolean"}
minterpolate_fps=60#@param {type:"integer"}
tblend_on = True #@param {type:"boolean"}
#tmix_on = True #@param {type:"boolean"}
frame_step_value = "2" #@param {type:"string"}


#deblock_toggle = True #@param {type:"boolean"}
################
#@markdown Set `deflicker_avg_frames` filter size in frames. FFmpeg's default is 5. I haven't tested too much but I would try 100.

#@markdown ---


#@markdown Note: Your output will look like `filename_outputStr.mp4` and can be found in your `compress_path`.

#@markdown Note: Right now only one visual effect can be enabled at a time. Tblend works best.


#if animation_tune_toggle:
#  tune_value = '-tune animation'

#video effects
if enable_visual_effects:
  visual_effects = '-vf'
  if minterpolate_on:
    visual_effects = visual_effects + ' minterpolate=fps=' + str(minterpolate_fps) + ':mi_mode=blend'
  if deflicker_on:
    visual_effects = visual_effects + ' deflicker=s=' + str(deflicker_avg_frames) + ':m=am'
  if mpdecimate_on:
    visual_effects = visual_effects + ' mpdecimate'
  if tblend_on:
    visual_effects = visual_effects + ' tblend=all_mode=average,framestep=' + frame_step_value
#  if tmix_on:
#    visual_effects = visual_effects + ' tmix=frames=5:weights=\"1 1 1 1 1\",select=\'not(mod(n\,5))\''
#  if deblock_toggle:
#    x265_params = '-x265-params deblock=-3,-3 '
else:
  visual_effects = ''



runFF()



ffmpeg version 3.4.8-0ubuntu0.2 Copyright (c) 2000-2020 the FFmpeg developers
  built with gcc 7 (Ubuntu 7.5.0-3ubuntu1~18.04)
  configuration: --prefix=/usr --extra-version=0ubuntu0.2 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --enable-gpl --disable-stripping --enable-avresample --enable-avisynth --enable-gnutls --enable-ladspa --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librubberband --enable-librsvg --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvorbis --enable-libvpx --enable-libwavpack --enable-libwebp --enable-libx265 --enable-libxml2 --enable-libxvid --enable-lib